# Calendar methods

This tutorial offers a walk through of all the `ExchangeCalendar` methods (for properties see the [calendar properties](./calendar_properties.ipynb) tutorial).

The following sections cover methods according to the nature of the argument(s) they take:
* [Methods that query a Date](#Methods-that-query-a-Date)  
* [Methods that query a Session](#Methods-that-query-a-Session)  
* [Methods that query a Minute](#Methods-that-query-a-Minute)  
* [Methods that query multiple TradingMinute](#Methods-that-query-multiple-TradingMinute)  
* [Methods that query a range of dates](#Methods-that-query-a-range-of-dates)

The following sections cover methods that evaluate an index of trading minutes or sessions:
* [Methods that evaluate an index of contiguous trading minutes](#Methods-that-evaluate-an-index-of-contiguous-trading-minutes)
* [Methods that evaluate an index of contiguous sessions](#Methods-that-evaluate-an-index-of-contiguous-sessions)

`Date` and `Session` refer to types for 'session' parameters (see [sessions.ipynb](./sessions.ipynb) for a tutorial on how to work with sessions).

`Minute` and `TradingMinute` refer to types for 'minute' parameters (see [minutes.ipynb](./minutes.ipynb) for a tutorial on how to work with mintues).

In [2]:
# setup
import exchange_calendars as xcals
import pandas as pd

one_minute = pd.Timedelta(1, "T")

nys = xcals.get_calendar("XNYS", side="left") # New York Stock Exchange
hkg = xcals.get_calendar("XHKG", side="left") # Hong Kong Stock Exchange

### Methods that query a Date

The argument of methods in this section takes a `Date` type.

In [3]:
# for reference, all times are UTC
nys.schedule.loc["2020-12-31":"2021-01-05"]

,market_open,break_start,break_end,market_close
2020-12-31 00:00:00+00:00,2020-12-31 14:30:00,NaT,NaT,2020-12-31 21:00:00
2021-01-04 00:00:00+00:00,2021-01-04 14:30:00,NaT,NaT,2021-01-04 21:00:00
2021-01-05 00:00:00+00:00,2021-01-05 14:30:00,NaT,NaT,2021-01-05 21:00:00


**`is_session`** queries if a date represents a session.

In [4]:
nys.is_session("2021-01-01"), nys.is_session("2021-01-04")

(False, True)

**`date_to_session`** will return the the passed `date` if `date` represents a session, or otherwise the closest session in the passed `direction`.

In [5]:
# date is a session so`direction` is ignored
nys.date_to_session("2021-01-04", direction="next")

Timestamp('2021-01-04 00:00:00+0000', tz='UTC')

In [6]:
nys.date_to_session("2021-01-01", direction="next")

Timestamp('2021-01-04 00:00:00+0000', tz='UTC', freq='C')

In [7]:
nys.date_to_session("2021-01-01", direction="previous")

Timestamp('2020-12-31 00:00:00+0000', tz='UTC', freq='C')

A ValueError is raised if `direction` is not passed and `date` is not a session.

In [ ]:
nys.date_to_session("2021-01-01")
# run cell for full traceback

```python
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
~\AppData\Local\Temp/ipykernel_17852/717127884.py in <module>
----> 1 nys.date_to_session("2021-01-01")

ValueError: `date` '2021-01-01 00:00:00+00:00' does not represent a session. Consider passing a `direction`.
```

### Methods that query a Session

The argument of methods in this section takes a `Session` type.

The following methods can be used to return the **open, close** and **break times** of a session...

In [9]:
# for reference, all times are UTC
session = "2021-01-04"
hkg.schedule.loc[[session]]

,market_open,break_start,break_end,market_close
2021-01-04 00:00:00+00:00,2021-01-04 01:30:00,2021-01-04 04:00:00,2021-01-04 05:00:00,2021-01-04 08:00:00


In [10]:
hkg.session_open(session), hkg.session_close(session)

(Timestamp('2021-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2021-01-04 08:00:00+0000', tz='UTC'))

In [11]:
# or
hkg.session_open_close(session)

(Timestamp('2021-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2021-01-04 08:00:00+0000', tz='UTC'))

In [12]:
hkg.session_break_start(session), hkg.session_break_end(session)

(Timestamp('2021-01-04 04:00:00+0000', tz='UTC'),
 Timestamp('2021-01-04 05:00:00+0000', tz='UTC'))

In [13]:
# or
hkg.session_break_start_end(session)

(Timestamp('2021-01-04 04:00:00+0000', tz='UTC'),
 Timestamp('2021-01-04 05:00:00+0000', tz='UTC'))

In [14]:
# in local time
hkg.session_open(session).tz_convert(hkg.tz)

Timestamp('2021-01-04 09:30:00+0800', tz='Asia/Hong_Kong')

The **`session_*_minute`** methods return a boundary trading minute for a session or subsession.

In [15]:
hkg.session_first_minute(session), hkg.session_last_minute(session)

(Timestamp('2021-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2021-01-04 07:59:00+0000', tz='UTC'))

In [16]:
hkg.session_last_am_minute(session), hkg.session_first_pm_minute(session)

(Timestamp('2021-01-04 03:59:00+0000', tz='UTC'),
 Timestamp('2021-01-04 05:00:00+0000', tz='UTC'))

In [17]:
# or get first and last minutes together...
hkg.session_first_last_minute(session)

(Timestamp('2021-01-04 01:30:00+0000', tz='UTC'),
 Timestamp('2021-01-04 07:59:00+0000', tz='UTC'))

In [18]:
# NB with "right" side...
hkg_right = xcals.get_calendar("XHKG", side="right")
hkg_right.session_first_last_minute(session)

(Timestamp('2021-01-04 01:31:00+0000', tz='UTC'),
 Timestamp('2021-01-04 08:00:00+0000', tz='UTC'))

**`session_has_break`** to query if a session has a break.

In [19]:
# Hong Kong has a break, at least on `session`...
hkg.session_has_break(session)

True

In [20]:
# but New York does not
nys.session_has_break(session)

False

In [21]:
# see...
nys.schedule.loc[[session]]

,market_open,break_start,break_end,market_close
2021-01-04 00:00:00+00:00,2021-01-04 14:30:00,NaT,NaT,2021-01-04 21:00:00


**`next_session`** and **`previous_session`** do what they say on the tin.

In [22]:
# just to recall...
session

'2021-01-04'

In [23]:
hkg.previous_session(session)

Timestamp('2020-12-31 00:00:00+0000', tz='UTC', freq='C')

In [24]:
hkg.next_session(session)

Timestamp('2021-01-05 00:00:00+0000', tz='UTC', freq='C')

**`session_offset`** takes a 'count' argument which provides for offsetting a session by a given number of sessions. If `count` is -1 or 1 then the method behaves as `previous_session` and `next_session` respectively.

In [25]:
nys.session_offset(session, count=-1), nys.session_offset(session, count=1)

(Timestamp('2020-12-31 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2021-01-05 00:00:00+0000', tz='UTC', freq='C'))

In [26]:
nys.session_offset(session, -2), nys.session_offset(session, 2)

(Timestamp('2020-12-30 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2021-01-06 00:00:00+0000', tz='UTC', freq='C'))

In [27]:
nys.session_offset(session, -50), nys.session_offset(session, 50)

(Timestamp('2020-10-21 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2021-03-17 00:00:00+0000', tz='UTC', freq='C'))

In [28]:
# if you must...
nys.session_offset(session, 0)

Timestamp('2021-01-04 00:00:00+0000', tz='UTC', freq='C')

**`session_minutes`** returns an index of all the trading minutes of the passed session.

In [29]:
hkg.session_minutes(session)

DatetimeIndex(['2021-01-04 01:30:00+00:00', '2021-01-04 01:31:00+00:00',
               '2021-01-04 01:32:00+00:00', '2021-01-04 01:33:00+00:00',
               '2021-01-04 01:34:00+00:00', '2021-01-04 01:35:00+00:00',
               '2021-01-04 01:36:00+00:00', '2021-01-04 01:37:00+00:00',
               '2021-01-04 01:38:00+00:00', '2021-01-04 01:39:00+00:00',
               ...
               '2021-01-04 07:50:00+00:00', '2021-01-04 07:51:00+00:00',
               '2021-01-04 07:52:00+00:00', '2021-01-04 07:53:00+00:00',
               '2021-01-04 07:54:00+00:00', '2021-01-04 07:55:00+00:00',
               '2021-01-04 07:56:00+00:00', '2021-01-04 07:57:00+00:00',
               '2021-01-04 07:58:00+00:00', '2021-01-04 07:59:00+00:00'],
              dtype='datetime64[ns, UTC]', length=330, freq=None)

### Methods that query a Minute

The argument of methods in this section takes a `Minute` type.

**`is_trading_minute`** returns a boolean indicating if a minute is a trading minute.

In [30]:
# for reference
hkg.schedule.loc[["2021-01-04"]]

,market_open,break_start,break_end,market_close
2021-01-04 00:00:00+00:00,2021-01-04 01:30:00,2021-01-04 04:00:00,2021-01-04 05:00:00,2021-01-04 08:00:00


In [31]:
hkg.is_trading_minute("2021-01-04 01:25")

False

In [32]:
hkg.is_trading_minute("2021-01-04 01:35")

True

In [33]:
break_minute = "2021-01-04 04:35"
hkg.is_trading_minute(break_minute)

False

**`is_break_minute`** returns a boolean indicating if a minute lies within a lunch break. Break minutes lie between the last trading minute of the morning subsession and first trading minute of the afternoon subsession, exclusive of both (a break minute cannot be a trading minute).

In [34]:
hkg.is_break_minute(break_minute)

True

**`is_open_on_minute`** can take an optional `include_breaks` argument. If `include_breaks` is False (the default) then the method behaves in the same way as `is_trading_minute`. If `include_breaks` is True then the method will return True if `minute` is either a trading minute or a break minute (otherwise returns False).

In [35]:
hkg.is_open_on_minute(break_minute)

False

In [36]:
hkg.is_open_on_minute(break_minute, ignore_breaks=True)

True

NB **From version 4.0 `is_open_on_minute` may change behaviour** to distinguish the method from `is_trading_minute`. See [#61](https://github.com/gerrymanoim/exchange_calendars/issues/61).

**`minute_to_trading_minute`** will return the passed `minute` if `minute` is a trading minute, or otherwise the closest minute in the passed `direction`.

In [37]:
# define a trading minute
trading_minute = nys.first_minutes["2010-10-27"] + pd.Timedelta(45, "T")
trading_minute, nys.is_trading_minute(trading_minute)

(Timestamp('2010-10-27 14:15:00+0000', tz='UTC'), True)

In [38]:
# minute is a trading_minute, so `direction` ignored
nys.minute_to_trading_minute(trading_minute, direction="next")

Timestamp('2010-10-27 14:15:00+0000', tz='UTC')

In [39]:
# define a non-trading minute
non_trading_minute = nys.first_minutes["2010-10-27"] - pd.Timedelta(3, "T")
non_trading_minute, nys.is_trading_minute(non_trading_minute)

(Timestamp('2010-10-27 13:27:00+0000', tz='UTC'), False)

In [40]:
nys.minute_to_trading_minute(non_trading_minute, direction="next")

Timestamp('2010-10-27 13:30:00+0000', tz='UTC')

In [41]:
nys.minute_to_trading_minute(non_trading_minute, direction="previous")

Timestamp('2010-10-26 19:59:00+0000', tz='UTC')

A ValueError is raised if `direction` is not passed and `minute` is not a trading minute.

In [ ]:
nys.minute_to_trading_minute(non_trading_minute)
# run cell for full traceback

```python
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
~\AppData\Local\Temp/ipykernel_17852/2461711297.py in <module>
----> 1 nys.minute_to_trading_minute(non_trading_minute)

ValueError: `minute` '2010-10-27 13:27:00+00:00' is not a trading minute. Consider passing `direction` as 'next' or 'previous'.
```

**`previous_open`**, **`next_open`**, **`previous_close`** and **`next_close`** will return the previous or next open or close relative to the passed `minute`. NB Which minutes are treated as trading minutes is irrelevant.

In [43]:
# for reference
nys.schedule.loc["2021-01-04":"2021-01-05"]

,market_open,break_start,break_end,market_close
2021-01-04 00:00:00+00:00,2021-01-04 14:30:00,NaT,NaT,2021-01-04 21:00:00
2021-01-05 00:00:00+00:00,2021-01-05 14:30:00,NaT,NaT,2021-01-05 21:00:00


In [44]:
open_05 = nys.session_open("2021-01-05")
open_05

Timestamp('2021-01-05 14:30:00+0000', tz='UTC')

In [45]:
nys.previous_open(open_05), nys.next_open(open_05)

(Timestamp('2021-01-04 14:30:00+0000', tz='UTC'),
 Timestamp('2021-01-06 14:30:00+0000', tz='UTC'))

In [46]:
nys.previous_open(open_05 + one_minute)

Timestamp('2021-01-05 14:30:00+0000', tz='UTC')

In [47]:
nys.previous_close(open_05), nys.next_close(open_05)

(Timestamp('2021-01-04 21:00:00+0000', tz='UTC'),
 Timestamp('2021-01-05 21:00:00+0000', tz='UTC'))

**`previous_minute`** and **`next_minute`** return the first trading minute prior to / following `minute`.

In [48]:
minute = nys.first_minutes["2021-01-05"]
minute

Timestamp('2021-01-05 14:30:00+0000', tz='UTC')

In [49]:
nys.previous_minute(minute), nys.next_minute(minute)

(Timestamp('2021-01-04 20:59:00+0000', tz='UTC'),
 Timestamp('2021-01-05 14:31:00+0000', tz='UTC'))

In [50]:
minute -= (one_minute * 2)
minute

Timestamp('2021-01-05 14:28:00+0000', tz='UTC')

In [51]:
nys.previous_minute(minute), nys.next_minute(minute)

(Timestamp('2021-01-04 20:59:00+0000', tz='UTC'),
 Timestamp('2021-01-05 14:30:00+0000', tz='UTC'))

**`minute_offset`** takes a 'count' argument which provides for offsetting a trading minute by the given number of minutes. If count is -1 or 1 then the method behaves as `previous_minute` and `next_minute` respectively.

In [52]:
# recalling
trading_minute

Timestamp('2010-10-27 14:15:00+0000', tz='UTC')

In [53]:
nys.minute_offset(trading_minute, count=-1), nys.minute_offset(trading_minute, 1)

(Timestamp('2010-10-27 14:14:00+0000', tz='UTC'),
 Timestamp('2010-10-27 14:16:00+0000', tz='UTC'))

In [54]:
nys.minute_offset(trading_minute, -5), nys.minute_offset(trading_minute, 5)

(Timestamp('2010-10-27 14:10:00+0000', tz='UTC'),
 Timestamp('2010-10-27 14:20:00+0000', tz='UTC'))

In [55]:
nys.minute_offset(trading_minute, -45), nys.minute_offset(trading_minute, 344)

(Timestamp('2010-10-27 13:30:00+0000', tz='UTC'),
 Timestamp('2010-10-27 19:59:00+0000', tz='UTC'))

In [56]:
nys.minute_offset(trading_minute, -46), nys.minute_offset(trading_minute, 345)

(Timestamp('2010-10-26 19:59:00+0000', tz='UTC'),
 Timestamp('2010-10-28 13:30:00+0000', tz='UTC'))

In [57]:
nys.minute_offset(trading_minute, -3000), nys.minute_offset(trading_minute, 3000)

(Timestamp('2010-10-15 16:15:00+0000', tz='UTC'),
 Timestamp('2010-11-05 18:45:00+0000', tz='UTC'))

In [58]:
# and of course
nys.minute_offset(trading_minute, 0)

Timestamp('2010-10-27 14:15:00+0000', tz='UTC')

**`minute_to_session`** returns the session associated with `minute`. If `minute` is a trading minute or a break minute then this will be the session of which the minute is a trading/break minute.

In [59]:
nys.minute_to_session(trading_minute)

Timestamp('2010-10-27 00:00:00+0000', tz='UTC', freq='C')

If `minute` is not a trading or break minute then the return is determined by the optional `direction` parameter. If `direction` is "next" the return will be the closest session after `minute`, whilst if `direction` is "previous" the return will be the closest session before `minute`.

In [60]:
# for reference
non_trading_minute

Timestamp('2010-10-27 13:27:00+0000', tz='UTC')

In [61]:
# default `direction` is "next"
nys.minute_to_session(non_trading_minute)

Timestamp('2010-10-27 00:00:00+0000', tz='UTC', freq='C')

In [62]:
nys.minute_to_session(non_trading_minute, direction="previous")

Timestamp('2010-10-26 00:00:00+0000', tz='UTC', freq='C')

`direction` can also take "none", in which case `minute` is required to be a trading or break minute. To the contrary a ValueError is raised:

In [ ]:
nys.minute_to_session(non_trading_minute, direction="none")
# run cell for full traceback

```python
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
~\AppData\Local\Temp/ipykernel_17852/1432708257.py in <module>
----> 1 nys.minute_to_session(non_trading_minute, direction="none")

ValueError: `minute` '2010-10-27 13:27:00+00:00' is not a trading minute. Consider passing `direction` as 'next' or 'previous'.
```

**`minute_to_past_session`** returns a session that closed before the given `minute`. If the passed `minute` is a trading minute then the method will NOT return the session of which `minute` is a trading minute, but rather a prior session.

In [64]:
# recalling
trading_minute

Timestamp('2010-10-27 14:15:00+0000', tz='UTC')

In [65]:
nys.minute_to_past_session(trading_minute)

Timestamp('2010-10-26 00:00:00+0000', tz='UTC', freq='C')

Note how the above differs to `minute_to_session`...

In [66]:
nys.minute_to_session(trading_minute)

Timestamp('2010-10-27 00:00:00+0000', tz='UTC', freq='C')

`minute_to_past_session` can take an optional `count` argument (default 1) to offset the past session returned.

In [67]:
# as default
nys.minute_to_past_session(trading_minute, count=1)

Timestamp('2010-10-26 00:00:00+0000', tz='UTC', freq='C')

In [68]:
nys.minute_to_past_session(trading_minute, count=2)

Timestamp('2010-10-25 00:00:00+0000', tz='UTC', freq='C')

In [69]:
nys.minute_to_past_session(trading_minute, count=3)
# jumping a weekend...

Timestamp('2010-10-22 00:00:00+0000', tz='UTC', freq='C')

**`minute_to_future_session`** works in the same way as **`minute_to_past_session`** albeit looking fowards.

In [70]:
nys.minute_to_future_session(trading_minute)
# default count is 1

Timestamp('2010-10-28 00:00:00+0000', tz='UTC', freq='C')

In [71]:
nys.minute_to_future_session(trading_minute, count=2)

Timestamp('2010-10-29 00:00:00+0000', tz='UTC', freq='C')

**`minute_offset_by_sessions`** offsets a trading minute by a given number of sessions, by default 1.

In [72]:
# recalling again
trading_minute

Timestamp('2010-10-27 14:15:00+0000', tz='UTC')

In [73]:
nys.minute_offset_by_sessions(trading_minute)
# default count is 1

Timestamp('2010-10-28 14:15:00+0000', tz='UTC', freq='C')

In [74]:
nys.minute_offset_by_sessions(trading_minute, count=-2)
# offseting 'backwards'

Timestamp('2010-10-25 14:15:00+0000', tz='UTC', freq='C')

If a would-be offset minute is not a minute of the offset session then...

If the would-be offset minute is later than the offset session's close, the offset session's last trading minute is returned.

In [75]:
# for reference
nys.schedule.loc["2020-12-23":"2020-12-28", "market_close"].to_frame()

,market_close
2020-12-23 00:00:00+00:00,2020-12-23 21:00:00
2020-12-24 00:00:00+00:00,2020-12-24 18:00:00
2020-12-28 00:00:00+00:00,2020-12-28 21:00:00


In [76]:
nys.minute_offset_by_sessions("2020-12-23 20:22", 1)

Timestamp('2020-12-24 17:59:00+0000', tz='UTC')

If the would-be offset minute is earlier than the offset session's open, the offset session's first trading minute is returned.

In [77]:
# for reference
hkg.schedule.loc["'2011-03-03":"2011-03-08", "market_open"].to_frame()

,market_open
2011-03-03 00:00:00+00:00,2011-03-03 02:00:00
2011-03-04 00:00:00+00:00,2011-03-04 02:00:00
2011-03-07 00:00:00+00:00,2011-03-07 01:30:00
2011-03-08 00:00:00+00:00,2011-03-08 01:30:00


In [78]:
hkg.minute_offset_by_sessions("2011-03-08 01:47", count=-2)

Timestamp('2011-03-04 02:00:00+0000', tz='UTC')

If the would-be offset minute is a break minute, the last pre-break trading minute is returned.

In [79]:
# for reference
hkg.schedule.loc["2010-12-23":"2010-12-24"]

,market_open,break_start,break_end,market_close
2010-12-23 00:00:00+00:00,2010-12-23 02:00:00,2010-12-23 04:00:00,2010-12-23 05:00:00,2010-12-23 08:00:00
2010-12-24 00:00:00+00:00,2010-12-24 02:00:00,NaT,NaT,2010-12-24 04:30:00


In [80]:
hkg.minute_offset_by_sessions("2010-12-24 04:15", count=-1)

Timestamp('2010-12-23 03:59:00+0000', tz='UTC')

### Methods that evaluate an index of contiguous trading minutes

**`minutes_in_range`** returns all trading minutes within and inclusive of `start_minute` and `end_minute`. The parameters take `Minute` type (i.e. they do not need to represent actual trading minutes).

In [81]:
# recalling...
hkg.schedule.loc[["2021-01-04"]]

,market_open,break_start,break_end,market_close
2021-01-04 00:00:00+00:00,2021-01-04 01:30:00,2021-01-04 04:00:00,2021-01-04 05:00:00,2021-01-04 08:00:00


In [82]:
hkg.minutes_in_range("2021-01-04 03:55", "2021-01-04 05:05")

DatetimeIndex(['2021-01-04 03:55:00+00:00', '2021-01-04 03:56:00+00:00',
               '2021-01-04 03:57:00+00:00', '2021-01-04 03:58:00+00:00',
               '2021-01-04 03:59:00+00:00', '2021-01-04 05:00:00+00:00',
               '2021-01-04 05:01:00+00:00', '2021-01-04 05:02:00+00:00',
               '2021-01-04 05:03:00+00:00', '2021-01-04 05:04:00+00:00',
               '2021-01-04 05:05:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [83]:
# `start_minute` does not have to represent a trading minute
hkg.minutes_in_range("2021-01-04 04:30", "2021-01-04 05:05")

DatetimeIndex(['2021-01-04 05:00:00+00:00', '2021-01-04 05:01:00+00:00',
               '2021-01-04 05:02:00+00:00', '2021-01-04 05:03:00+00:00',
               '2021-01-04 05:04:00+00:00', '2021-01-04 05:05:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

Alternatively, **`minutes_window`** can be used to create an index of trading minutes of a defined length, anchored by the passed `minute`.

In [84]:
hkg.minutes_window("2021-01-04 07:55", count=9)

DatetimeIndex(['2021-01-04 07:55:00+00:00', '2021-01-04 07:56:00+00:00',
               '2021-01-04 07:57:00+00:00', '2021-01-04 07:58:00+00:00',
               '2021-01-04 07:59:00+00:00', '2021-01-05 01:30:00+00:00',
               '2021-01-05 01:31:00+00:00', '2021-01-05 01:32:00+00:00',
               '2021-01-05 01:33:00+00:00', '2021-01-05 01:34:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

Or to work backwards from `minute`...

In [85]:
hkg.minutes_window("2021-01-04 07:55", count=-9)

DatetimeIndex(['2021-01-04 07:46:00+00:00', '2021-01-04 07:47:00+00:00',
               '2021-01-04 07:48:00+00:00', '2021-01-04 07:49:00+00:00',
               '2021-01-04 07:50:00+00:00', '2021-01-04 07:51:00+00:00',
               '2021-01-04 07:52:00+00:00', '2021-01-04 07:53:00+00:00',
               '2021-01-04 07:54:00+00:00', '2021-01-04 07:55:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

**NB** The index will have length one greater than the `count` (this behaviour may change in release 4.0, see [#61](https://github.com/gerrymanoim/exchange_calendars/issues/61)).

### Methods that query multiple TradingMinute

**`minutes_to_sessions`** does for multiple TradingMinute what `minute_to_session` does for one. However, this method is limited by requiring every minute of the passed `minutes` parameter to be a trading minute (i.e. there's no `direction` parameter available for this one). Also, whilst minutes do not need to be contiguous, they do need to be in ascending order.

In [86]:
# set up a minute index to pass as `minutes`..
ser = nys.first_minutes[-200:-195] + (one_minute * 5)
ser

2022-01-07 00:00:00+00:00   2022-01-07 14:35:00+00:00
2022-01-10 00:00:00+00:00   2022-01-10 14:35:00+00:00
2022-01-11 00:00:00+00:00   2022-01-11 14:35:00+00:00
2022-01-12 00:00:00+00:00   2022-01-12 14:35:00+00:00
2022-01-13 00:00:00+00:00   2022-01-13 14:35:00+00:00
Freq: C, Name: first_minutes, dtype: datetime64[ns, UTC]

In [87]:
# still setting up 'minutes'
mins = pd.DatetimeIndex(ser)
mins = mins[:1].union(mins[-2:], sort=False)
mins

DatetimeIndex(['2022-01-07 14:35:00+00:00', '2022-01-12 14:35:00+00:00',
               '2022-01-13 14:35:00+00:00'],
              dtype='datetime64[ns, UTC]', name='first_minutes', freq=None)

In [88]:
nys.minutes_to_sessions(mins)

DatetimeIndex(['2022-01-07 00:00:00+00:00', '2022-01-12 00:00:00+00:00',
               '2022-01-13 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [89]:
# NB `minutes` can also be passed as a Series...
nys.minutes_to_sessions(ser)

DatetimeIndex(['2022-01-07 00:00:00+00:00', '2022-01-10 00:00:00+00:00',
               '2022-01-11 00:00:00+00:00', '2022-01-12 00:00:00+00:00',
               '2022-01-13 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

### Methods that evaluate an index of contiguous sessions

**`sessions_in_range`** does for sessions what `minutes_in_range` does for minutes. Returns an index of all the sessions between `start` and `end`. Parameters take `Date` type (i.e. they do not need to represent actual sessions).

In [90]:
nys.sessions_in_range("2020", "2020-12-31")

DatetimeIndex(['2020-01-02 00:00:00+00:00', '2020-01-03 00:00:00+00:00',
               '2020-01-06 00:00:00+00:00', '2020-01-07 00:00:00+00:00',
               '2020-01-08 00:00:00+00:00', '2020-01-09 00:00:00+00:00',
               '2020-01-10 00:00:00+00:00', '2020-01-13 00:00:00+00:00',
               '2020-01-14 00:00:00+00:00', '2020-01-15 00:00:00+00:00',
               ...
               '2020-12-17 00:00:00+00:00', '2020-12-18 00:00:00+00:00',
               '2020-12-21 00:00:00+00:00', '2020-12-22 00:00:00+00:00',
               '2020-12-23 00:00:00+00:00', '2020-12-24 00:00:00+00:00',
               '2020-12-28 00:00:00+00:00', '2020-12-29 00:00:00+00:00',
               '2020-12-30 00:00:00+00:00', '2020-12-31 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=253, freq='C')

**`sessions_window`** holds no surprises if you're familiar with `minutes_window`...

In [91]:
nys.sessions_window("2020-12-23", 3)

DatetimeIndex(['2020-12-23 00:00:00+00:00', '2020-12-24 00:00:00+00:00',
               '2020-12-28 00:00:00+00:00', '2020-12-29 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='C')

In [92]:
nys.sessions_window("2020-12-23", -3)

DatetimeIndex(['2020-12-18 00:00:00+00:00', '2020-12-21 00:00:00+00:00',
               '2020-12-22 00:00:00+00:00', '2020-12-23 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='C')

**NB** As for `minutes_window`, the length of the returned index is one greater than `count` (...and again this may change with release 4.0, see [#61](https://github.com/gerrymanoim/exchange_calendars/issues/61))

### Methods that query a range of dates

The methods in this section query sessions that fall within the range of dates from `start` through `end` (inclusive of both). Both parameters take a `Date` (i.e the passed values can but do not have to represent an actual session).

In [93]:
start, end = "2021-12-23", "2021-12-29"
# for reference
nys.schedule.loc[start:end]

,market_open,break_start,break_end,market_close
2021-12-23 00:00:00+00:00,2021-12-23 14:30:00,NaT,NaT,2021-12-23 21:00:00
2021-12-27 00:00:00+00:00,2021-12-27 14:30:00,NaT,NaT,2021-12-27 21:00:00
2021-12-28 00:00:00+00:00,2021-12-28 14:30:00,NaT,NaT,2021-12-28 21:00:00
2021-12-29 00:00:00+00:00,2021-12-29 14:30:00,NaT,NaT,2021-12-29 21:00:00


**`sessions_distance`** returns the number of sessions in the range.

In [94]:
nys.sessions_distance(start, end)

4

**`sessions_minutes`** returns an index comprised of the trading minutes of all sessions in the given range.

In [95]:
nys.sessions_minutes(start, end)

DatetimeIndex(['2021-12-23 14:30:00+00:00', '2021-12-23 14:31:00+00:00',
               '2021-12-23 14:32:00+00:00', '2021-12-23 14:33:00+00:00',
               '2021-12-23 14:34:00+00:00', '2021-12-23 14:35:00+00:00',
               '2021-12-23 14:36:00+00:00', '2021-12-23 14:37:00+00:00',
               '2021-12-23 14:38:00+00:00', '2021-12-23 14:39:00+00:00',
               ...
               '2021-12-29 20:50:00+00:00', '2021-12-29 20:51:00+00:00',
               '2021-12-29 20:52:00+00:00', '2021-12-29 20:53:00+00:00',
               '2021-12-29 20:54:00+00:00', '2021-12-29 20:55:00+00:00',
               '2021-12-29 20:56:00+00:00', '2021-12-29 20:57:00+00:00',
               '2021-12-29 20:58:00+00:00', '2021-12-29 20:59:00+00:00'],
              dtype='datetime64[ns, UTC]', length=1560, freq=None)

**`sessions_minutes_count`** returns just the number of trading minutes corresponding to the sessions.

In [96]:
nys.sessions_minutes_count(start, end)

1560

In [97]:
# i.e. `sessions_minutes_count` behaves as...
len(nys.sessions_minutes(start, end))

1560

**`sessions_opens`** and **`sessions_closes`** return pd.Series describing open/close times over the session range.

In [98]:
nys.sessions_opens(start, end)

2021-12-23 00:00:00+00:00   2021-12-23 14:30:00+00:00
2021-12-27 00:00:00+00:00   2021-12-27 14:30:00+00:00
2021-12-28 00:00:00+00:00   2021-12-28 14:30:00+00:00
2021-12-29 00:00:00+00:00   2021-12-29 14:30:00+00:00
Freq: C, Name: market_open, dtype: datetime64[ns, UTC]

NB the return differs from the following only in that **`sessions_opens`** returns as tz "UTC"

In [99]:
nys.opens[start:end]

2021-12-23 00:00:00+00:00   2021-12-23 14:30:00
2021-12-27 00:00:00+00:00   2021-12-27 14:30:00
2021-12-28 00:00:00+00:00   2021-12-28 14:30:00
2021-12-29 00:00:00+00:00   2021-12-29 14:30:00
Freq: C, Name: market_open, dtype: datetime64[ns]

...and following release 4.0 there will be no difference, on which basis it's been **proposed that `sessions_opens` and `sessions_closes` are removed in 4.0** ([#61](https://github.com/gerrymanoim/exchange_calendars/issues/61)).

### **`trading_index`**  
Saving the best for last, `trading_index` provides for creating a trading index of given `period` over a range of sessions. It's options provide so much flexibility that it's got it's own tutorial at [trading_index.ipynb](./trading_index.ipynb). 

Here you get but a mere taster...

In [100]:
# recalling
start, end

('2021-12-23', '2021-12-29')

In [101]:
nys.trading_index(start, end, "15min", intervals=False)

DatetimeIndex(['2021-12-23 14:30:00+00:00', '2021-12-23 14:45:00+00:00',
               '2021-12-23 15:00:00+00:00', '2021-12-23 15:15:00+00:00',
               '2021-12-23 15:30:00+00:00', '2021-12-23 15:45:00+00:00',
               '2021-12-23 16:00:00+00:00', '2021-12-23 16:15:00+00:00',
               '2021-12-23 16:30:00+00:00', '2021-12-23 16:45:00+00:00',
               ...
               '2021-12-29 18:30:00+00:00', '2021-12-29 18:45:00+00:00',
               '2021-12-29 19:00:00+00:00', '2021-12-29 19:15:00+00:00',
               '2021-12-29 19:30:00+00:00', '2021-12-29 19:45:00+00:00',
               '2021-12-29 20:00:00+00:00', '2021-12-29 20:15:00+00:00',
               '2021-12-29 20:30:00+00:00', '2021-12-29 20:45:00+00:00'],
              dtype='datetime64[ns, UTC]', length=104, freq=None)

In [102]:
nys.trading_index(start, start, "24min", closed="right", intervals=False)

DatetimeIndex(['2021-12-23 14:54:00+00:00', '2021-12-23 15:18:00+00:00',
               '2021-12-23 15:42:00+00:00', '2021-12-23 16:06:00+00:00',
               '2021-12-23 16:30:00+00:00', '2021-12-23 16:54:00+00:00',
               '2021-12-23 17:18:00+00:00', '2021-12-23 17:42:00+00:00',
               '2021-12-23 18:06:00+00:00', '2021-12-23 18:30:00+00:00',
               '2021-12-23 18:54:00+00:00', '2021-12-23 19:18:00+00:00',
               '2021-12-23 19:42:00+00:00', '2021-12-23 20:06:00+00:00',
               '2021-12-23 20:30:00+00:00', '2021-12-23 20:54:00+00:00',
               '2021-12-23 21:18:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [103]:
nys.trading_index(start, start, "24min", force_close=True, intervals=False)

DatetimeIndex(['2021-12-23 14:30:00+00:00', '2021-12-23 14:54:00+00:00',
               '2021-12-23 15:18:00+00:00', '2021-12-23 15:42:00+00:00',
               '2021-12-23 16:06:00+00:00', '2021-12-23 16:30:00+00:00',
               '2021-12-23 16:54:00+00:00', '2021-12-23 17:18:00+00:00',
               '2021-12-23 17:42:00+00:00', '2021-12-23 18:06:00+00:00',
               '2021-12-23 18:30:00+00:00', '2021-12-23 18:54:00+00:00',
               '2021-12-23 19:18:00+00:00', '2021-12-23 19:42:00+00:00',
               '2021-12-23 20:06:00+00:00', '2021-12-23 20:30:00+00:00',
               '2021-12-23 20:54:00+00:00', '2021-12-23 21:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [104]:
index = nys.trading_index(start, end, "20min", intervals=True)
index

IntervalIndex([[2021-12-23 14:30:00, 2021-12-23 14:50:00), [2021-12-23 14:50:00, 2021-12-23 15:10:00), [2021-12-23 15:10:00, 2021-12-23 15:30:00), [2021-12-23 15:30:00, 2021-12-23 15:50:00), [2021-12-23 15:50:00, 2021-12-23 16:10:00) ... [2021-12-29 19:30:00, 2021-12-29 19:50:00), [2021-12-29 19:50:00, 2021-12-29 20:10:00), [2021-12-29 20:10:00, 2021-12-29 20:30:00), [2021-12-29 20:30:00, 2021-12-29 20:50:00), [2021-12-29 20:50:00, 2021-12-29 21:10:00)],
              closed='left',
              dtype='interval[datetime64[ns, UTC]]')

In [105]:
# just to see it clearer...
pd.DataFrame(dict(left_side=index.left, right_side=index.right), index=index)

,left_side,right_side
"[2021-12-23 14:30:00, 2021-12-23 14:50:00)",2021-12-23 14:30:00+00:00,2021-12-23 14:50:00+00:00
"[2021-12-23 14:50:00, 2021-12-23 15:10:00)",2021-12-23 14:50:00+00:00,2021-12-23 15:10:00+00:00
"[2021-12-23 15:10:00, 2021-12-23 15:30:00)",2021-12-23 15:10:00+00:00,2021-12-23 15:30:00+00:00
"[2021-12-23 15:30:00, 2021-12-23 15:50:00)",2021-12-23 15:30:00+00:00,2021-12-23 15:50:00+00:00
"[2021-12-23 15:50:00, 2021-12-23 16:10:00)",2021-12-23 15:50:00+00:00,2021-12-23 16:10:00+00:00
...,...,...
"[2021-12-29 19:30:00, 2021-12-29 19:50:00)",2021-12-29 19:30:00+00:00,2021-12-29 19:50:00+00:00
"[2021-12-29 19:50:00, 2021-12-29 20:10:00)",2021-12-29 19:50:00+00:00,2021-12-29 20:10:00+00:00
"[2021-12-29 20:10:00, 2021-12-29 20:30:00)",2021-12-29 20:10:00+00:00,2021-12-29 20:30:00+00:00
"[2021-12-29 20:30:00, 2021-12-29 20:50:00)",2021-12-29 20:30:00+00:00,2021-12-29 20:50:00+00:00
